# Imports

We have to set the tensorflow version using `tensorflow_version 1.x`

In [ ]:
%tensorflow_version 1.x

import tensorflow as tf

TensorFlow 1.x selected.


In [ ]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools
!pip install tf_slim
!pip install numpy==1.17.0

Selecting previously unselected package python-bs4.
(Reading database ... 160980 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

In [ ]:
%cd /models/research

/gdrive/MyDrive/Projects/Pedestrian_Detection/models/research


In [ ]:
# !pwd
# !protoc object_detection/protos/*.proto --python_out=.

/content/drive/MyDrive/Projects/Pedestrian_Detection/models/research


In [ ]:
%cd /models/research/slim/

import os
os.environ['PYTHONPATH'] += '/content/drive/MyDrive/Projects/Pedestrian_Detection/models/research:/content/drive/MyDrive/Projects/Pedestrian_Detection/models/research/slim/'

!python setup.py build
!python setup.py install

/gdrive/MyDrive/Projects/Pedestrian_Detection/models/research/slim
running build
running build_py
running egg_info
writing slim.egg-info/PKG-INFO
writing dependency_links to slim.egg-info/dependency_links.txt
writing requirements to slim.egg-info/requires.txt
writing top-level names to slim.egg-info/top_level.txt
writing manifest file 'slim.egg-info/SOURCES.txt'
running install
running bdist_egg
running egg_info
writing slim.egg-info/PKG-INFO
writing dependency_links to slim.egg-info/dependency_links.txt
writing requirements to slim.egg-info/requires.txt
writing top-level names to slim.egg-info/top_level.txt
writing manifest file 'slim.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/datasets
copying build/lib/datasets/imagenet_2012_validation_synset_labels.txt -> build/bdist.linux-x86_64/egg/datasets
copying build/lib/datasets/imagenet.py -> bui

## Importing Libraries

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
#from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [ ]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [ ]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

import math
import itertools
from itertools import compress
from PIL import Image, ImageDraw

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [ ]:
#!mkdir /content/drive/MyDrive/Projects/Pedestrian_Detection/models/research/pretrained_model
%cd /models/research/pretrained_model

# !wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
# !tar -xzf ssd_mobilenet_v2_coco_2018_03_29.tar.gz -C .

%cd models/research

/gdrive/MyDrive/Projects/Pedestrian_Detection/models/research/pretrained_model
/gdrive/MyDrive/Projects/Pedestrian_Detection/models/research


In [ ]:
# Model Name
MODEL_NAME = 'ssd_mobilenet_v2_coco_2018_03_29'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = '/models/research/pretrained_model/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/models/research/object_detection/data/mscoco_label_map.pbtxt'

## Load a (frozen) Tensorflow model into memory.

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/models/research/object_detection/test_images'
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

Function to filter predictions by label. We only want display `person`, in this case `person` is `id=1`. We can check it in `category_index`

In [ ]:
category_index

{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suitc

In [ ]:
def filter_boxes(min_score, boxes, scores, classes, categories):
  """Return boxes with a confidence >= `min_score`"""
  n = len(classes)
  idxs = []
  for i in range(n):
      if classes[i] in categories and scores[i] >= min_score:
          idxs.append(i)
  
  filtered_boxes = boxes[idxs, ...]
  filtered_scores = scores[idxs, ...]
  filtered_classes = classes[idxs, ...]
  return filtered_boxes, filtered_scores, filtered_classes

When we get the prediction from Tensorflow, we get a list of boxes with all predictions. This function allows us to get the coordinates of a box.

In [ ]:
def calculate_coord(bbox, width, height):
    xmin = bbox[1] * width
    ymin = bbox[0] * height
    xmax = bbox[3] * width
    ymax = bbox[2] * height

    return [xmin, ymin, xmax - xmin, ymax - ymin]

Function to calculate the centroid

In [ ]:
def calculate_centr(coord):
  return (coord[0]+(coord[2]/2), coord[1]+(coord[3]/2))

Calculate distance between 2 centroids

In [ ]:
def calculate_centr_distances(centroid_1, centroid_2):
  return  math.sqrt((centroid_2[0]-centroid_1[0])**2 + (centroid_2[1]-centroid_1[1])**2)

Calculate all permutations between the centroids

In [ ]:
def calculate_perm(centroids):
  permutations = []
  for current_permutation in itertools.permutations(centroids, 2):
    if current_permutation[::-1] not in permutations:
      permutations.append(current_permutation)
  return permutations

Calculate the middle point between 2 points

In [ ]:
def midpoint(p1, p2):
    return ((p1[0] + p2[0])/2, (p1[1] + p2[1])/2)

Calculate the slope of a segment

In [ ]:
def calculate_slope(x1, y1, x2, y2):
    m = (y2-y1)/(x2-x1)
    return m

Main function to predict pedestrians and calculate the distance between them

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from random import randrange

def show_inference(image_path):
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)

  # Get boxes only for person
  confidence_cutoff = 0.5
  boxes, scores, classes = filter_boxes(confidence_cutoff, output_dict['detection_boxes'], output_dict['detection_scores'], output_dict['detection_classes'], [1])

  # Get width and heigth
  im = Image.fromarray(image_np)
  width, height = im.size

  # Pixel per meters - THIS IS A REFERENCE, YOU HAVE TO ADAPT THIS FOR EACH IMAGE
  # In this case, we are considering that (width - 150) approximately is 7 meters
  average_px_meter = (width-150) / 7

  # Calculate normalized coordinates for boxes
  centroids = []
  coordinates = []
  for box in boxes:
    coord = calculate_coord(box, width, height)
    centr = calculate_centr(coord)
    centroids.append(centr)
    coordinates.append(coord)

  # Calculate all permutations
  permutations = calculate_perm(centroids)

  # Display boxes and centroids
  fig, ax = plt.subplots(figsize = (20,12), dpi = 90)
  ax.imshow(image, interpolation='nearest')
  for coord, centr in zip(coordinates, centroids):
    ax.add_patch(patches.Rectangle((coord[0], coord[1]), coord[2], coord[3], linewidth=2, edgecolor='y', facecolor='none', zorder=10))
    ax.add_patch(patches.Circle((centr[0], centr[1]), 3, color='yellow', zorder=20))

  # Display lines between centroids
  for perm in permutations:
    dist = calculate_centr_distances(perm[0], perm[1])
    dist_m = dist/average_px_meter

    print("M meters: ", dist_m)
    middle = midpoint(perm[0], perm[1])
    print("Middle point", middle)

    x1 = perm[0][0]
    x2 = perm[1][0]
    y1 = perm[0][1]
    y2 = perm[1][1]

    slope = calculate_slope(x1, y1, x2, y2)
    dy = math.sqrt(3**2/(slope**2+1))
    dx = -slope*dy

    # Display randomly the position of our distance text
    if randrange(10) % 2== 0:
      Dx = middle[0] - dx*10
      Dy = middle[1] - dy*10
    else:
      Dx = middle[0] + dx*10
      Dy = middle[1] + dy*10

    if dist_m < 1.5:
      ax.annotate("{}m".format(round(dist_m, 2)), xy=middle, color='white', xytext=(Dx, Dy), fontsize=10, arrowprops=dict(arrowstyle='->', lw=1.5, color='yellow'), bbox=dict(facecolor='red', edgecolor='white', boxstyle='round', pad=0.2), zorder=30)
      ax.plot((perm[0][0], perm[1][0]), (perm[0][1], perm[1][1]), linewidth=2, color='yellow', zorder=15)
    elif 1.5 < dist_m < 3.5:
      ax.annotate("{}m".format(round(dist_m, 2)), xy=middle, color='black', xytext=(Dx, Dy), fontsize=8, arrowprops=dict(arrowstyle='->', lw=1.5, color='skyblue'), bbox=dict(facecolor='y', edgecolor='white', boxstyle='round', pad=0.2), zorder=30)
      ax.plot((perm[0][0], perm[1][0]), (perm[0][1], perm[1][1]), linewidth=2, color='skyblue', zorder=15)
    else:
      pass

Uncomment show_inference() and add image path ==> show_inference('img.jpg')

In [ ]:
# Make prediction
#show_inference('')

M meters:  3.7905392563844456
Middle point (602.978935203515, 443.427863240242)
M meters:  6.065934537251108
Middle point (989.2125168954954, 753.6752786636353)
M meters:  1.1491598110480408
Middle point (323.3983964314684, 679.1379255056381)
M meters:  3.6833364974628497
Middle point (1418.1121798455715, 478.0501900911331)
M meters:  2.7188373353844293
Middle point (752.2980593815446, 403.512836933136)
M meters:  4.981214333427742
Middle point (1138.531641073525, 713.7602523565292)


# Videos

In [ ]:
import cv2
import matplotlib
from matplotlib import pyplot as plt
plt.ioff()
matplotlib.use('Agg')

FILE_OUTPUT = '/Projects/TEST/Predicted.mp4'

# Playing video from file
cap = cv2.VideoCapture('/models/research/object_detection/test_images/WhatsApp Video 2021-03-21 at 3.56.07 PM.mp4')

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
width = int(cap.get(3))
height = int(cap.get(4))

dim = (width, height)
print(dim)

tope = 10
i = 0
new = True
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        i = 0
        while(cap.isOpened()):
            # Capture frame-by-frame
            ret, frame = cap.read()
            
            if ret == True:
              # Correct color
              frame = gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

              # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
              image_np_expanded = np.expand_dims(frame, axis=0)

              # Actual detection.
              (boxes, scores, classes, num) = sess.run(
                  [detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
              
              # Filter boxes
              confidence_cutoff = 0.5
              boxes, scores, classes = filter_boxes(confidence_cutoff, np.squeeze(boxes), np.squeeze(scores), np.squeeze(classes), [1])

              # Calculate normalized coordinates for boxes
              centroids = []
              coordinates = []
              for box in boxes:
                coord = calculate_coord(box, width, height)
                centr = calculate_centr(coord)
                centroids.append(centr)
                coordinates.append(coord)

              # Pixel per meters
              average_px_meter = (width-150) / 7

              permutations = calculate_perm(centroids)

              # Display boxes and centroids
              fig, ax = plt.subplots(figsize = (20,12), dpi = 90, frameon=False)
              ax = fig.add_axes([0, 0, 1, 1])
              ax.axis('off')
              ax.spines['top'].set_visible(False)
              ax.spines['right'].set_visible(False)
              ax.spines['bottom'].set_visible(False)
              ax.spines['left'].set_visible(False)
              ax.get_xaxis().set_ticks([])
              ax.get_yaxis().set_ticks([])
              for coord, centr in zip(coordinates, centroids):
                ax.add_patch(patches.Rectangle((coord[0], coord[1]), coord[2], coord[3], linewidth=2, edgecolor='y', facecolor='none', zorder=10))
                ax.add_patch(patches.Circle((centr[0], centr[1]), 3, color='yellow', zorder=20))

              # Display lines between centroids
              for perm in permutations:
                dist = calculate_centr_distances(perm[0], perm[1])
                dist_m = dist/average_px_meter
                
                x1 = perm[0][0]
                y1 = perm[0][1]
                x2 = perm[1][0]
                y2 = perm[1][1]

                # Calculate middle point
                middle = midpoint(perm[0], perm[1])

                # Calculate slope
                slope = calculate_slope(x1, y1, x2, y2)
                dy = math.sqrt(3**2/(slope**2+1))
                dx = -slope*dy

                # Set random location
                if randrange(10) % 2== 0:
                  Dx = middle[0] - dx*10
                  Dy = middle[1] - dy*10
                else:
                  Dx = middle[0] + dx*10
                  Dy = middle[1] + dy*10

                if dist_m < 1.5:
                  ax.annotate("{}m".format(round(dist_m, 2)), xy=middle, color='white', xytext=(Dx, Dy), fontsize=10, arrowprops=dict(arrowstyle='->', lw=1.5, color='yellow'), bbox=dict(facecolor='red', edgecolor='white', boxstyle='round', pad=0.2), zorder=35)
                  ax.plot((perm[0][0], perm[1][0]), (perm[0][1], perm[1][1]), linewidth=2, color='yellow', zorder=15)
                elif 1.5 < dist_m < 3.5:
                  ax.annotate("{}m".format(round(dist_m, 2)), xy=middle, color='black', xytext=(Dx, Dy), fontsize=8, arrowprops=dict(arrowstyle='->', lw=1.5, color='skyblue'), bbox=dict(facecolor='y', edgecolor='white', boxstyle='round', pad=0.2), zorder=35)
                  ax.plot((perm[0][0], perm[1][0]), (perm[0][1], perm[1][1]), linewidth=2, color='skyblue', zorder=15)
                else:
                  pass
              
              ax.imshow(frame, interpolation='nearest')
              
              # This allows you to save each frame in a folder
              #fig.savefig("/gdrive/My Drive/Projects/Pedestrian_Detection/models/research/test_images/TEST_{}.png".format(i))
              # i += 1

              # Convert figure to numpy
              fig.canvas.draw()

              img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
              img  = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))

              img = np.array(fig.canvas.get_renderer()._renderer)
              img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)

              if new:
                print("Define out")
                out = cv2.VideoWriter(FILE_OUTPUT, cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (img.shape[1], img.shape[0]))
                new = False

              out.write(img)
            else:
              break

    # When everything done, release the video capture and video write objects
    cap.release()
    out.release()

    # Closes all the frames
    cv2.destroyAllWindows()

(1920, 1080)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:130: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


Define out


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
